In [1]:
!pip install transformers==4.31.0
!pip install -U bitsandbytes
!pip install accelerate
!pip install peft==0.4.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
import json
data=json.load(open("/content/recipes.json",'r')) #recipes webscraped dataset

In [4]:
new_data=[]
for x in data:
  temp="<s>[INST]"+x['Name']+"[/INST]"
  for y in x:
    if y!="Name":
      temp+=str(x[y])
  temp+="</s>"
  new_data.append(temp)

In [5]:
temp

'<s>[INST]Apple Fritters[/INST]DessertAmerican[\'2 cups all-purpose flour\', \'1/4 cup plus 2 tablespoons granulated sugar\', \'2 1/4 teaspoons baking powder\', \'1 teaspoon coarse salt\', \'3/4 cup whole milk\', \'2 large eggs, room temperature\', \'2 tablespoons melted unsalted butter\', \'1/2 teaspoon pure vanilla extract\', \'2 sweet apples, such as Rome, McIntosh, or Gala, peeled, cored, and cut into 1/4-inch pieces (about 3 cups)\', \'Safflower oil, for frying\', "Confectioners\' sugar, for dusting"][\'In a large bowl, whisk together flour, granulated sugar, baking powder, and salt. In another bowl, whisk together milk, eggs, butter, and vanilla. Gently fold milk mixture into flour mixture until just combined. Fold in apples.\', \'Meanwhile, heat 2 inches of oil in a medium heavy-bottomed pot over medium-high heat until a thermometer registers 350 degrees. Set a wire rack in a rimmed baking sheet.\', "Working in batches, drop heaping tablespoons of dough into oil. (A small ice cr

In [6]:
new_data={"text":new_data}

In [7]:
output_dir = "./results"

In [8]:
import pandas as pd
pd.DataFrame(new_data).to_csv("/content/new_data.csv")

In [9]:
from datasets import load_dataset
new_data=load_dataset("csv",data_files="/content/new_data.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
new_data

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 1876
})

In [ ]:
dataset = new_data

compute_dtype = getattr(torch, "bfloat16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant = False,
)
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and True:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
access_token = "*****" #add your access token
model_id = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token, timeout=60)

# Load model with 4-bit quantization and float16 compute dtype
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
    )
    use_auth_token=access_token,
    device_map="auto",  # Let transformers handle device placement
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained( "meta-llama/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=20,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    #max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=256,
)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1876 [00:00<?, ? examples/s]

In [13]:
model.train()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.

In [15]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035


In [16]:
for name, param in model.named_parameters():
    if "lora" in name:  # Check LoRA layers
        print(f"{name}: requires_grad={param.requires_grad}")

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight: requires_grad=True
base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight: requires_grad=True
base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight: requires_grad=True
base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight: requires_grad=True
base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight: requires_grad=True
base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight: requires_grad=True
base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight: requires_grad=True

In [17]:
torch.compile(model)

OptimizedModule(
  (_orig_mod): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096, padding_idx=0)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear(
                  in_features=4096, out_features=4096, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=64, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                )
                (k_proj): Lin

In [18]:
# Train model
trainer.train()

Step,Training Loss
25,2.344700
50,2.331400
75,2.329700


TrainOutput(global_step=93, training_loss=2.3347886198310444, metrics={'train_runtime': 485.213, 'train_samples_per_second': 3.866, 'train_steps_per_second': 0.192, 'total_flos': 1.8331787325825024e+16, 'train_loss': 2.3347886198310444, 'epoch': 0.99})

In [20]:
trainer.model.save_pretrained("cookbook_model")